In [1]:
import pandas as pd

try:
    questions = pd.read_csv('questions_clean.csv')
    websites = pd.read_csv('websites.csv')
    sample_submission = pd.read_csv('sample_submission.csv')
    print("Вопросы:")
    print(questions.head(2))
    print("\nБаза знаний (сайты):")
    print(websites.head(2))
    
except FileNotFoundError:
    print("Ошибка: Убедитесь, что файлы Questions.csv и Websites.csv находятся в той же папке.")

Вопросы:
   q_id                  query
0     1            Номер счета
1     2  Где узнать бик и счёт

База знаний (сайты):
   web_id                          url  kind  \
0       1         https://alfabank.ru/  html   
1       2  https://alfabank.ru/a-club/  html   

                                               title  \
0  Альфа-Банк - кредитные и дебетовые карты, кред...   
1                      А-Клуб. Деньги имеют значение   

                                                text  
0  Рассчитайте выгоду\nРасчёт калькулятора предва...  
1  Брокерские услуги\nОткрытие брокерского счёта ...  


In [2]:
# Этот код вставляем после загрузки websites.csv

# -- Начало нового блока (Чанкование) --

def get_chunks(text, chunk_size=256, overlap=64):
    """Простая функция для нарезки текста на слова."""
    words = text.split()
    if not words:
        return []
    
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk_words = words[i:i + chunk_size]
        chunks.append(" ".join(chunk_words))
    return chunks

chunk_data = [] # Это будет наша новая "база знаний"

print("Начинаем процесс чанкования...")

# Итерируемся по каждой строке в websites
for index, row in websites.iterrows():
    web_id = row['web_id']
    title = row['title'] if pd.notna(row['title']) else ''
    text = row['text'] if pd.notna(row['text']) else ''
    
    # Мы добавим заголовок к каждому чанку, это часто улучшает релевантность
    # (модель будет знать, из какого раздела этот текст)
    text_with_title = f"Заголовок: {title}\nТекст: {text}"
    
    chunks = get_chunks(text_with_title, chunk_size=256, overlap=64)
    
    for chunk_text in chunks:
        chunk_data.append({
            'web_id': web_id,  # Сохраняем, какому 'web_id' принадлежит чанк
            'text': chunk_text
        })

print(f"Готово. Получили {len(chunk_data)} чанков из {len(websites)} документов.")

# Создадим DataFrame из чанков для удобства
# (Это необязательно, но так проще)
chunks_df = pd.DataFrame(chunk_data)

# -- Конец нового блока --

Начинаем процесс чанкования...
Готово. Получили 9902 чанков из 1937 документов.


In [3]:
from sentence_transformers import SentenceTransformer

# 1. Загружаем модель
# 'paraphrase-multilingual-MiniLM-L12-v2' - хорошая, быстрая модель
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2') 

# 2. Векторизуем базу знаний (сайты)
# Убедимся, что нет пропусков (NaN), иначе эмбеддер выдаст ошибку
print("Начинаем векторизацию корпуса (чанков)... Это займет больше времени.")
# Передаем список текстов из наших чанков
corpus_embeddings = model.encode(
    chunks_df['text'].tolist(), 
    show_progress_bar=True
)

# Векторизация вопросов (этот блок остается БЕЗ ИЗМЕНЕНИЙ):
questions['query'] = questions['query'].fillna('')
print("Начинаем векторизацию вопросов...")
query_embeddings = model.encode(
    questions['query'].tolist(), 
    show_progress_bar=True
)

print(f"Форма эмбеддингов корпуса (чанков): {corpus_embeddings.shape}")
print(f"Форма эмбеддингов вопросов: {query_embeddings.shape}")

Начинаем векторизацию корпуса (чанков)... Это займет больше времени.


Batches:   0%|          | 0/310 [00:00<?, ?it/s]

Начинаем векторизацию вопросов...


Batches:   0%|          | 0/219 [00:00<?, ?it/s]

Форма эмбеддингов корпуса (чанков): (9902, 384)
Форма эмбеддингов вопросов: (6977, 384)


In [4]:
# Этот код остается таким же, он просто будет работать с эмбеддингами чанков
import faiss
import numpy as np

corpus_embeddings = corpus_embeddings.astype('float32')
query_embeddings = query_embeddings.astype('float32')

faiss.normalize_L2(corpus_embeddings)
faiss.normalize_L2(query_embeddings)

d = corpus_embeddings.shape[1]
index = faiss.IndexFlatIP(d)    

index.add(corpus_embeddings)

print(f"Индекс создан. Всего векторов в базе (чанков): {index.ntotal}")

Индекс создан. Всего векторов в базе (чанков): 9902


In [5]:
# Этот код полностью ЗАМЕНЯЕТ старый Шаг 5
k_candidates = 50
print("Начинаем агрегацию (голосование) по web_id...")
D, I = index.search(query_embeddings, k_candidates)
# Получаем массив `web_id` из нашего DataFrame чанков
# Порядок web_id здесь соответствует порядку эмбеддингов в faiss
chunk_web_ids_array = chunks_df['web_id'].values

# Получаем массив `q_id` из questions
q_ids_array = questions['q_id'].values

results_list = []

# Итерируемся по каждому вопросу
for i in range(len(q_ids_array)):
    q_id = q_ids_array[i]
    
    # I[i] - это список из 50 *индексов чанков*
    top_chunk_indices = I[i]
    
    # Находим web_id, которые соответствуют этим чанкам
    top_web_ids = chunk_web_ids_array[top_chunk_indices]
    
    # --- Логика голосования ---
    # Считаем, как часто встречается каждый web_id
    # Используем pd.Series для быстрого подсчета
    # value_counts() автоматически сортирует по убыванию частоты
    web_id_votes = pd.Series(top_web_ids).value_counts()
    
    # Берем топ-5 web_id по "голосованию"
    top_5_web_ids = web_id_votes.head(5).index.tolist()
    
    # ---
    
    # Добавляем их в наш список для submit
    for web_id in top_5_web_ids:
        results_list.append({'q_id': q_id, 'web_id': web_id})
        
# Создаем финальный DataFrame
submit_df = pd.DataFrame(results_list)

# Сохраняем в CSV
submit_df.to_csv('submiDDDDDDDDDDDD.csv', index=False)

print("\nФайл submit.csv (с чанкованием и голосованием) успешно создан!")
print(submit_df.head(10))

Начинаем агрегацию (голосование) по web_id...

Файл submit.csv (с чанкованием и голосованием) успешно создан!
   q_id  web_id
0     1    1705
1     1    1704
2     1     372
3     1    1157
4     1    1758
5     2     741
6     2     372
7     2     699
8     2      25
9     2       1


In [6]:
def check_my_question(query_text, k=5):
    print(f"Поисковый запрос: '{query_text}'\n")
    
    # 1. Векторизуем ОДИН наш вопрос (по той же технологии)
    query_vector = model.encode([query_text]).astype('float32')
    
    # 2. Нормализуем его
    faiss.normalize_L2(query_vector)
    
    # 3. Ищем в индексе (faiss) k лучших чанков
    # Ищем k_candidates, чтобы потом проголосовать
    k_candidates = 50 
    D, I = index.search(query_vector, k_candidates)
    
    # I - это 2D массив, [[idx1, idx2, ...]], нам нужен первый элемент I[0]
    top_chunk_indices = I[0] 
    
    # Получаем массив web_id из chunks_df
    chunk_web_ids_array = chunks_df['web_id'].values
    chunk_texts_array = chunks_df['text'].values
    
    # 4. Находим web_id, которые соответствуют этим чанкам
    top_web_ids = chunk_web_ids_array[top_chunk_indices]
    
    # 5. Голосование
    web_id_votes = pd.Series(top_web_ids).value_counts()
    
    # Берем топ-k web_id по "голосованию"
    top_k_web_ids = web_id_votes.head(k).index.tolist()
    
    print(f"--- Найдено {k} лучших web_id (по голосованию): ---\n")
    
    for i, web_id in enumerate(top_k_web_ids):
        print(f"Место #{i+1}: web_id = {web_id} (встретился {web_id_votes.iloc[i]} раз)")
        
        # Найдем пример чанка, который привел к этому web_id
        # (Это немного сложно, но полезно для отладки)
        try:
            # Находим *первый* чанк в топ-50, который относится к этому web_id
            first_match_index_in_top_50 = np.where(top_web_ids == web_id)[0][0]
            original_chunk_index = top_chunk_indices[first_match_index_in_top_50]
            
            chunk_text_example = chunk_texts_array[original_chunk_index]
            print(f"  Пример релевантного чанка: \n  '{chunk_text_example[:300]}...'\n")
        except Exception as e:
            print(f"  Не удалось найти пример чанка (ошибка: {e})\n")

# --- Конец блока для проверки ---


# --- ПРИМЕР ИСПОЛЬЗОВАНИЯ ---
# Вызовите эту функцию с любым вопросом
# (После того как ваш скрипт создал 'model', 'index' и 'chunks_df')

check_my_question("Как какать??", k=5)
check_my_question("Кредит, кредит, альфа?", k=5)

Поисковый запрос: 'Как какать??'

--- Найдено 5 лучших web_id (по голосованию): ---

Место #1: web_id = 1705 (встретился 2 раз)
  Пример релевантного чанка: 
  '1 .................................................................................................................................................... 135 Приложение № 2 .....................................................................................................................................'

Место #2: web_id = 1704 (встретился 2 раз)
  Пример релевантного чанка: 
  'ПОПОЛНЕНИЯ И ЧАСТИЧНОГО СНЯТИЯ). 10.1.Депозит «Альфа-Вклад Ключевой» (без опций пополнения и частичного снятия) - срочный депозит с возможностью выбора условия начисления процентов с Капитализацией или без Капитализации. 10.2.Депозит «Альфа-Вклад Ключевой» (без опций пополнения и частичного снятия) ...'

Место #3: web_id = 1259 (встретился 1 раз)
  Пример релевантного чанка: 
  'на переводы?...'

Место #4: web_id = 1089 (встретился 1 раз)
  Пример релев

In [7]:
df1 = pd.read_csv('submiDDDDDDDDDDDD.csv')
df2 = pd.read_csv('submit.csv')
sample_submission = pd.read_csv('sample_submission.csv')
print(df1.head(10))
print("FFFFFFFFFFFFFFFFFFFFF")
print(df2.head(10))
print("FFFFFFFFFFFFFFFFFFFFF")
print(sample_submission.head(10))
print("FFFFFFFFFFFFFFFFFFFFF")

   q_id  web_id
0     1    1705
1     1    1704
2     1     372
3     1    1157
4     1    1758
5     2     741
6     2     372
7     2     699
8     2      25
9     2       1
FFFFFFFFFFFFFFFFFFFFF
   q_id  web_id
0     1     372
1     1     334
2     1    1721
3     1    1159
4     1     741
5     2     372
6     2       1
7     2      25
8     2     741
9     2     699
FFFFFFFFFFFFFFFFFFFFF
   q_id                      web_list
0     1   [935, 687, 963, 1893, 1885]
1     2  [1660, 1882, 1040, 1090, 64]
2     3    [1246, 698, 465, 757, 307]
3     4    [366, 1625, 461, 279, 678]
4     5  [898, 589, 1162, 1889, 1181]
5     6     [299, 43, 1919, 313, 751]
6     7    [806, 247, 778, 668, 1674]
7     8    [336, 515, 398, 368, 1606]
8     9  [1461, 1490, 959, 1782, 564]
9    10  [158, 1140, 1512, 693, 1698]
FFFFFFFFFFFFFFFFFFFFF


In [10]:
print(questions.head(10))

   q_id                                              query
0     1                                        Номер счета
1     2                              Где узнать бик и счёт
2     3  Мне не приходят коды для подтверждения данной ...
3     4  Оформила рассрочку ,но уведомлений никаких не ...
4     5  Здравствуйте, когда смогу пользоваться кредитн...
5     6                  По истории платеж не отображается
6     7  Здравствуйте, в рп. Нахабино отделение банка н...
7     8                            Узнать зарплатный счет 
8     9  Подскажите, где я могу увидеть дату начала пол...
9    10                                    Не приходит смс


In [27]:
def transform_submission_format(df):
    """
    Преобразует из формата:
    q_id web_id
    1    372
    1    334
    1    1721
    ...
    в формат:
    q_id web_list
    1    [372, 334, 1721, ...]
    """
    # Группируем по q_id и собираем web_id в списки
    result = df.groupby('q_id')['web_id'].apply(list).reset_index()
    result.columns = ['q_id', 'web_list']
    
    return result

In [32]:

df = pd.read_csv('submit1.csv')
df.head()

,q_id,web_id
0,2692,938
1,2692,1118
2,2692,1516
3,2692,1552
4,2692,1553


In [34]:
df = pd.read_csv('submission.csv')
df.head()

,q_id,web_id
0,1,372
1,1,1157
2,1,334
3,1,1721
4,1,1775


In [30]:
transform_submission_format(df)

,q_id,web_list
0,1,"[372, 1705, 1704, 1157, 334]"
1,2,"[1157, 343, 479, 372, 363]"
2,3,"[1705, 1704, 1762, 1721]"
3,4,"[1705, 1704, 1758, 1753, 1760]"
4,5,"[164, 1029, 927, 176, 172]"
...,...,...
6572,6973,"[1704, 1705, 659, 1651, 1495]"
6573,6974,"[1249, 1157, 773, 775, 705]"
6574,6975,"[1704, 1721, 398, 1159, 851]"
6575,6976,"[1704, 1616, 858, 859, 1774]"


In [35]:
df.to_csv('submit.csv', index=False)